# Collab Write Analysis dataset

In [70]:
import pandas as pd
import itertools

fill_with_r1 = False
nan_if_multilabels = True

## Data Exploration and cleaning

In [71]:
# read the data (format xlsm)
data = pd.read_excel('data/CollabWriteAnalysisCountCodesLadyorTigerF20nS21S22wGSAnalysis27Jan2023_14Mar2024CleanF.xlsm', sheet_name='Combined Discussions')

In [72]:
# retrieve the columns
data.columns

Index(['Course', 'Book ID', 'Topic', 'Bookclub', 'Chat0CREW1', 'Pseudonym',
       'Message', 'Message Time', 'Is Answer', 'Page', 'Response Number',
       'R1 Discussion type', 'R2DiscussionType',
       'R2DiscussionTypeInterpNothers', 'CollapsR2DiscussionTypeInterpNothers',
       'Chat0CREW1B', 'R1 Dialogic spell', 'BinaryR1DialogicSpell',
       'R1 Uptake', 'BinaryR1Uptake', 'R2DialogicSpell',
       'BinaryR2DialogicSpell', 'R2Uptake', 'BinaryR2Uptake', 'Pseudonym.1',
       'Message.1', 'Bookclub.1', 'R1 Question', 'R2 Question', 'R1 Pivot',
       'R2 Pivot', 'Memo', 'Old Code Book'],
      dtype='object')

In [73]:
columns_to_delete = []

# Find the columns which content is equal to each other (and erase them)
for column, column2 in itertools.combinations(data.columns, 2):
    if data[column].equals(data[column2]):
        print(f'{column} is equal to {column2}')
        columns_to_delete.append(column2)

# Delete the columns
data.drop(columns=columns_to_delete, inplace=True)

Bookclub is equal to Bookclub.1
Chat0CREW1 is equal to Chat0CREW1B
Pseudonym is equal to Pseudonym.1
Message is equal to Message.1
BinaryR1DialogicSpell is equal to BinaryR1Uptake
BinaryR1DialogicSpell is equal to BinaryR2Uptake
BinaryR1Uptake is equal to BinaryR2Uptake


Value count:

In [74]:
for column in data.columns:
    if column in ['Message', 'Pseudonym', 'Message Time']:
        continue
    # value counts
    print(data[column].value_counts())
    
    # print the number of items not nan
    print(data[column].count())
    print()

Course
PIM P1                    452
? S22 Chat Synch Data      98
Outreach P1                53
? S22 Chat Asynch Data      8
Name: count, dtype: int64
611

Book ID
260    421
261    190
Name: count, dtype: int64
611

Topic
Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?                                                                                                                                                                                                               519
Part 1: Discuss what would happens next and why, providing evidence or logic. What is behind the door, the lady of the tiger?  Part 2: Later, the King discovers that his daughter had broken the law, by first finding out what was behind the doors & then signaling to her lover which door to open. What would the king do, and why? Do NOT write fiction, but rather speculate and

In [75]:
# Memo, old code book, topic are not needed
data.drop(columns=['Memo', 'Old Code Book', 'Topic'], inplace=True)

In [76]:
# remove the first ? if present
data['Course'] = data['Course'].apply(lambda x: x[2:] if x[0] == '?' else x)

In [77]:
# if Is Answer is nan or ' ' then it is True, otherwise False
data['Is Answer'] = data['Is Answer'].isna() | data['Is Answer'].str.isspace()

In [78]:
str_to_int = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
# change bookclub to the number of book club (format Book Club n)
data['Bookclub'] = data['Bookclub'].map(lambda x: str_to_int[x.split(' ')[-1]])

In [79]:
# Page number is NaN if ' ' or nan or ? 
data['Page'] = data['Page'].map(lambda x: pd.NA if str(x) in ['?'] or pd.isna(x) or str(x).isspace() else x)

In [80]:
# response number is 0 if ' '
data['Response Number'] = data['Response Number'].map(lambda x: pd.NA if str(x).isspace() or pd.isna(x) else x)

Rename classes:

In [81]:
# rename Book ID and Chat0CREW1 to book_id and chat_crew
columns_to_rename = {'Book ID': 'book_id', 
                     'Chat0CREW1': 'chat_crew',
                     'Bookclub': 'bookclub',
                     'Course': 'course',
                     'Pseudonym': 'pseudonym',
                     'Message': 'message',
                     'Message Time': 'time',
                     'Is Answer': 'is_answer', 
                     'Page': 'page', 
                     'Response Number': 'response_number', 
                     'R1 Discussion type': 'r1_discussion_type',
                     'R2DiscussionType': 'r2_discussion_type',
                     'R2DiscussionTypeInterpNothers': 'r2_discussion_type_interp_nothers',
                     'CollapsR2DiscussionTypeInterpNothers': 'collaps_r2_discussion_type_interp_nothers',
                     'R1 Dialogic spell': 'r1_dialogic_spell',
                     'BinaryR1DialogicSpell': 'binary_r1_dialogic_spell',
                     'R1 Uptake': 'r1_uptake',
                     'BinaryR1Uptake': 'binary_r1_uptake',
                     'R2DialogicSpell': 'r2_dialogic_spell',
                     'BinaryR2DialogicSpell': 'binary_r2_dialogic_spell',
                     'R2Uptake': 'r2_uptake',
                     'BinaryR2Uptake': 'binary_r2_uptake',
                     'R1 Question': 'r1_question',
                     'R2 Question': 'r2_question',
                     'R1 Pivot': 'r1_pivot',
                     'R2 Pivot': 'r2_pivot'
                     }
data.rename(columns=columns_to_rename, inplace=True)

In [82]:
if fill_with_r1:
    # copy R1 to R2 if R2 is nan
    for column in data.columns:
        if column.startswith('r2'):
            corr_r1_column = column.replace('r2', 'r1')
            if corr_r1_column in data.columns:
                data[column] = data[column].fillna(data[corr_r1_column])

                # if r2 contains a question mark, then take the value from r1
                for i, row in data.iterrows():
                    if '?' in str(row[column]):
                        data.at[i, column] = row[corr_r1_column]

Drop R1 and binary classes

In [83]:
# remove r1 columns and binary columns
data = data[[column for column in data.columns if not column.startswith('r1')]]
data = data[[column for column in data.columns if not column.startswith('binary')]]

Check the values of all the classes:

In [84]:
for column in data.columns:
    if column in ['message', 'pseudonym', 'time']:
        continue
    # value counts
    print(data[column].value_counts(dropna=False))
    
    # print the number of items not nan
    print(data[column].count())
    print()

course
PIM P1                  452
S22 Chat Synch Data      98
Outreach P1              53
S22 Chat Asynch Data      8
Name: count, dtype: int64
611

book_id
260    421
261    190
Name: count, dtype: int64
611

bookclub
1    277
4    192
2     94
3     30
5     18
Name: count, dtype: int64
611

chat_crew
1    421
0    190
Name: count, dtype: int64
611

is_answer
False    503
True     108
Name: count, dtype: int64
611

page
10      334
<NA>    102
9        96
13       46
8        32
3         1
Name: count, dtype: int64
509

response_number
<NA>    191
3.4      97
2.4      94
3.2      63
2.1      60
2.2      24
3.5      18
1.1      18
1.3      12
2.3      12
3.1       9
1.2       7
3.3       6
Name: count, dtype: int64
420

r2_discussion_type
Seminar                  333
Deliberation              85
Social                    69
UX                        47
Procedure                 46
Imaginative entry         17
Other                      6
Seminar, Deliberation      2
Imaginative     

Creating a mapping from discussion type to collaps discussion type interp nothers:

In [85]:
discussion_to_collaps = {}
for index, row in data.iterrows():
    if row['r2_discussion_type'] not in discussion_to_collaps:
        discussion_to_collaps[row['r2_discussion_type']] = row['collaps_r2_discussion_type_interp_nothers']
    else:
        # check if the value is the same
        if discussion_to_collaps[row['r2_discussion_type']] != row['collaps_r2_discussion_type_interp_nothers']:
            print(f'Error in {row["r2_discussion_type"]}, {discussion_to_collaps[row["r2_discussion_type"]]} != {row["collaps_r2_discussion_type_interp_nothers"]}')

These are probable errors in the collaps. I will mantain the mapping.

In [86]:
data.drop(columns=['collaps_r2_discussion_type_interp_nothers', 'r2_discussion_type_interp_nothers'], inplace=True)
discussion_to_collaps

{'Social': 'Social/Procedural/UX',
 'Seminar': 'Interpretation',
 'Procedure': 'Social/Procedural/UX',
 'Other': 'Other',
 'Deliberation': 'Deliberation',
 'UX': 'Social/Procedural/UX',
 'Imaginative entry': 'Other',
 'Seminar, Deliberation': 'Interpretation',
 'Social, Deliberation': 'Social/Procedural/UX',
 'Deliberation, Seminar': 'Deliberation',
 'Social, Procedure': 'Social/Procedural/UX',
 'Imaginative': 'Other',
 'Imaginative Entry': 'Deliberation'}

### Class cleaning

Split the messages or duplicate them if they contain more classes (best split is based on '\n', '.', '?', '!', ';', ',', ' '; it is not possible to split only by successive split_char, so the order matter).

In [87]:
def check_class(data_to_check, column, possible_splits = ['\n', '.', '?', '!', ';', ',', ' '], divide_also =[], take_last_of = [], change_to = pd.NA):
    new_data = data_to_check.copy()

    rows_added = 0
    rows_without_splitting = 0
    
    # I want to see if the r2_discussion_type is a list of discussion types
    rows_to_check = new_data[new_data[column].notna() & new_data[column].str.contains(',')].iterrows()

    for index, row in rows_to_check:
        types = row[column].split(',')
        n_types = len(types)

        # remove multiple spaces
        message = [' '.join(row['message'].split())]

        for split_char in possible_splits:
            message = [item for message_part in message for item in message_part.split(split_char) if message_part]
            
            # if both the parts are not empty and the number of parts is equal to the number of types
            if len(message) == n_types and all(message_part for message_part in message):
                # I have found the right split, add rows to the data and remove the old row
                data_to_insert = pd.DataFrame(columns=new_data.columns)
                data_index = index + rows_added

                for type_index, (type_, message_part) in enumerate(zip(types, message)):
                    data_to_insert.loc[len(data_to_insert.index)] = row
                    data_to_insert.loc[data_to_insert.index[-1], column] = type_.strip()
                    data_to_insert.loc[data_to_insert.index[-1], 'message'] = message_part

                    if type_index < n_types - 1:
                        for column_to_remove in take_last_of:
                            data_to_insert.loc[data_to_insert.index[-1], column_to_remove] = pd.NA

                    for column_to_check in divide_also:
                        other_column = new_data.loc[data_index, column_to_check]
                        if other_column and not pd.isna(other_column):
                            other_column = str(other_column).split(',')
                            if len(other_column) == n_types:
                                data_to_insert.loc[data_to_insert.index[-1], column_to_check] = other_column[type_index].strip()

                    rows_added += 1
                
                new_data = pd.concat([new_data.loc[:data_index - 1], data_to_insert, new_data.loc[data_index + 1:]]).reset_index(drop=True)
                rows_added -= 1
                break
        else:
            # check if all the types are the same
            if len(set(types)) != 1:
                if nan_if_multilabels:
                    new_data.loc[index + rows_added, column] = change_to
                else:
                    rows_without_splitting += 1
                    # if I did not find the right split, duplicate the row and remove the multiple types
                    data_to_insert = pd.DataFrame(columns=new_data.columns)
                    data_index = index + rows_added
                    for type_index, type_ in enumerate(types):
                        data_to_insert.loc[len(data_to_insert.index)] = row
                        data_to_insert.loc[data_to_insert.index[-1], column] = type_.strip()

                        if type_index < n_types - 1:
                            for column_to_remove in take_last_of:
                                data_to_insert.loc[data_to_insert.index[-1], column_to_remove] = pd.NA

                        for column_to_check in divide_also:
                            other_column = new_data.loc[data_index, column_to_check]
                            if other_column and not pd.isna(other_column):
                                other_column = str(other_column).split(',')
                                if len(other_column) == n_types:
                                    data_to_insert.loc[data_to_insert.index[-1], column_to_check] = other_column[type_index].strip()

                        rows_added += 1

                    new_data = pd.concat([new_data.loc[:data_index - 1], data_to_insert, new_data.loc[data_index + 1:]]).reset_index(drop=True)
                    rows_added -= 1
            else:
                # remove the multiple types
                new_data.loc[index, column] = types[0].strip()
                
    return new_data

Discussion Type

In [88]:
data = check_class(data, 'r2_discussion_type', divide_also=['r2_question'], take_last_of=['r2_pivot'], change_to = 'Other')

wrong_types = {'Imaginative entry' : 'Imaginative Entry',
               'Imaginative': 'Imaginative Entry'}

data['r2_discussion_type'] = data['r2_discussion_type'].replace(wrong_types)

# strip the values
data['r2_discussion_type'] = data['r2_discussion_type'].str.strip()

print(data['r2_discussion_type'].value_counts(dropna=False))

r2_discussion_type
Seminar              335
Deliberation          87
Social                70
Procedure             47
UX                    47
Imaginative Entry     20
Other                  8
Name: count, dtype: int64


Dialogic spell

In [89]:
# Page number is NaN if ' ' or nan or ? 
data['r2_dialogic_spell'] = data['r2_dialogic_spell'].map(lambda x: pd.NA if str(x) in ['?', '\xa0'] or pd.isna(x) or str(x).isspace() else x)

# change Begin to 1 if not NA
data['r2_dialogic_spell'] = data['r2_dialogic_spell'].map(lambda x: 1 if pd.notna(x) and x == 'Begin' else x)

for i, row in data.iterrows():
    if pd.notna(row['r2_dialogic_spell']) and row['r2_dialogic_spell'] in ['End', 'Break']:
        data.at[i, 'r2_dialogic_spell'] = data.at[i - 1, 'r2_dialogic_spell']

# change everything to integers
data['r2_dialogic_spell'] = data['r2_dialogic_spell'].map(lambda x: int(x) if pd.notna(x) else x)

print(data['r2_dialogic_spell'].value_counts(dropna=False))

r2_dialogic_spell
1       200
<NA>    150
2       126
3        65
4        29
7        21
5        13
6        10
Name: count, dtype: int64


Uptake:

In [90]:
data['r2_uptake'] = data['r2_uptake'].map(lambda x: pd.NA if str(x) in ['?', '\xa0'] or pd.isna(x) or str(x).isspace() else x)

print(data['r2_uptake'].value_counts(dropna=False))

r2_uptake
<NA>         243
Affirm       142
Elaborate    108
Filler        61
Clarify       50
Disagree      10
Name: count, dtype: int64


Questions

In [91]:
data = check_class(data, 'r2_question', possible_splits=['?'], take_last_of=['r2_pivot'])

right_types = ['C-HOT', 'O-HOT', 'C-LOT', 'O-LOT']

# pd.NA if ' ' or nan or ?
data['r2_question'] = data['r2_question'].map(lambda x: pd.NA if pd.isna(x) or str(x) in ['?', '\xa0'] or str(x).isspace() or '?' in str(x) or str(x) not in right_types else x)

print(data['r2_question'].value_counts(dropna=False))

r2_question
<NA>     536
C-LOT     37
C-HOT     22
O-HOT     11
O-LOT      8
Name: count, dtype: int64


Pivot

In [92]:
# create from pivot column
data['r2_pivot'] = data['r2_pivot'].map(lambda x: pd.NA if str(x) in ['?', '\xa0'] or pd.isna(x) or str(x).isspace() else x)

# Make not possible to have a pivot with more to
data['r2_pivot'] = data['r2_pivot'].map(lambda x: pd.NA if pd.isna(x) or len(x.split(' to ')) > 2 else x)

data['from_pivot'] = data['r2_pivot'].map(lambda x: pd.NA if pd.isna(x) else x.split(' to ')[0].strip())
data['to_pivot'] = data['r2_pivot'].map(lambda x: pd.NA if pd.isna(x) else x.split(' to ')[1].strip())

misspellings = {
    'Imaginative' : 'Imaginative Entry',
    'Imaginative entry' : 'Imaginative Entry',
    'Delibration' : 'Deliberation',
    'Deliberationa' : 'Deliberation',
    'Deliberation/Social/Procedure/UX' : 'Social/Procedure/UX',
    'Socal' : 'Social/Procedure/UX',
}

data['from_pivot'] = data['from_pivot'].replace(misspellings)
data['to_pivot'] = data['to_pivot'].replace(misspellings)
data.drop(columns=['r2_pivot'], inplace=True)

print(data['from_pivot'].value_counts(dropna=False))
print()
print(data['to_pivot'].value_counts(dropna=False))

from_pivot
<NA>                   567
Seminar                 20
Social/Procedure/UX     15
Deliberation             9
Imaginative Entry        3
Name: count, dtype: int64

to_pivot
<NA>                   567
Social/Procedure/UX     18
Seminar                 16
Deliberation            10
Imaginative Entry        3
Name: count, dtype: int64


In [93]:
misplaced = 0
corrected = 0

small_collaps = {
    'Imaginative Entry': 'Imaginative Entry',
    'Deliberation': 'Deliberation',
    'Social' : 'Social/Procedure/UX',
    'Procedure' : 'Social/Procedure/UX',
    'UX' : 'Social/Procedure/UX',
    'Seminar' : 'Seminar',
    'Other': 'Other'
}

for index, row in data.iterrows():
    if pd.notna(row['from_pivot']) and pd.notna(row['to_pivot']):
        current_type = row['r2_discussion_type']

        previous_row = data.loc[index - 1]
        previous_type = previous_row['r2_discussion_type']

        if previous_type not in small_collaps or current_type not in small_collaps:
            continue

        if row['from_pivot'] == small_collaps[previous_row['r2_discussion_type']] and row['to_pivot'] == small_collaps[row['r2_discussion_type']]:
            corrected += 1
        else:
            from_pivot = row['from_pivot']
            to_pivot = row['to_pivot']
            print(f'Misplaced {from_pivot} to {to_pivot}')

            guess_from_pivot = small_collaps[previous_row["r2_discussion_type"]]
            guess_to_pivot = small_collaps[row["r2_discussion_type"]]

            print(f'Guessed {guess_from_pivot} to {guess_to_pivot}')
            print()
            misplaced += 1

print(f'Corrected: {corrected}, Misplaced: {misplaced}')

Misplaced Seminar to Social/Procedure/UX
Guessed Seminar to Seminar

Misplaced Social/Procedure/UX to Seminar
Guessed Seminar to Social/Procedure/UX

Misplaced Social/Procedure/UX to Deliberation
Guessed Deliberation to Deliberation

Misplaced Seminar to Deliberation
Guessed Seminar to Other

Misplaced Seminar to Social/Procedure/UX
Guessed Deliberation to Seminar

Misplaced Deliberation to Seminar
Guessed Social/Procedure/UX to Seminar

Corrected: 41, Misplaced: 6


### Save and try history

In [94]:
data.columns

Index(['course', 'book_id', 'bookclub', 'chat_crew', 'pseudonym', 'message',
       'time', 'is_answer', 'page', 'response_number', 'r2_discussion_type',
       'r2_dialogic_spell', 'r2_uptake', 'r2_question', 'from_pivot',
       'to_pivot'],
      dtype='object')

In [95]:
columns_to_predict = ['r2_discussion_type', 'r2_dialogic_spell', 'r2_uptake', 'r2_question', 'from_pivot', 'to_pivot']

In [101]:
# Discussion type without history
discussion_data = data.copy()
columns_to_drop = [column for column in columns_to_predict if column != 'r2_discussion_type']
discussion_data.drop(columns= columns_to_drop, inplace=True)

# copy only message and r2_discussion_type
discussion_data = discussion_data[['message', 'r2_discussion_type']]
discussion_data.rename(columns={'r2_discussion_type': 'y',
                                'message': 'X'
                                }, inplace=True)

# save to csv
discussion_data.to_csv('cleaned_data/discussion_data.csv', index=False)

## Plots